In [1]:
import torch.nn as nn
import tensorflow as tf
import numpy as np
import pandas as pd
from typing import Dict
import json
import urllib
# import torchvision
# import torchvision.transforms as transforms
import torch
import os
#To install certain libraries that will be used in pytorchvideo
MODEL = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True)
import torchvision.io as io
from sklearn.preprocessing import StandardScaler
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
    UniformCropVideo
)
from pytorchvideo.models import slowfast


Downloading: "https://github.com/facebookresearch/pytorchvideo/zipball/main" to C:\Users\salma/.cache\torch\hub\main.zip


ModuleNotFoundError: No module named 'fvcore'

In [6]:


class ModifiedSlowFast(nn.Module):
    def __init__(self, original_model):
        super(ModifiedSlowFast, self).__init__()

        # Copy all layers except the head from the original model
        self.features = nn.Sequential(*list(original_model.blocks.children())[:-1])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # Global average pooling

    def forward(self, x):
        # Forward pass through the modified SlowFast model
        x = self.features(x)
        x = self.avgpool(x)
        return x

In [7]:
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 1
frames_per_second = 30
slowfast_alpha = 4
num_clips = 10
num_crops = 3

class PackPathway(torch.nn.Module):
    """
    Transform for converting video frames as a list of tensors.
    """
    def __init__(self):
        super().__init__()

    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        # Perform temporal sampling from the fast pathway.
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // slowfast_alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list

transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(
                size=side_size
            ),
            CenterCropVideo(crop_size),
            PackPathway()
        ]
    ),
)

# The duration of the input clip is also specific to the model.
clip_duration = (num_frames * sampling_rate)/frames_per_second
device = 'cpu'

In [13]:
class TemporalDetection:
  def __init__(self, video_path, output_dir):
    self.model_feature_extraction_slowfast = torch.load('slowfast_1sec_extractore.h5')
    self.video_path = video_path
    self.MLP_Model = tf.keras.models.load_model('best_model_standarized.h5')
    self.video = EncodedVideo.from_path( video_path )
    self.output_dir = output_dir
  def extract_features(self, ):
    # model_feature_extraction_slowfast = torch.load('/content/drive/MyDrive/Tics Dataset/SlowfastTrial/models/slowfast_1sec_extractore.h5')
    # video = EncodedVideo.from_path( video_path )
    num_of_clips= int(self.video.duration) // int(1.0666666666666667)
    print("Video clips: ", num_of_clips)
    all_features = np.zeros((num_of_clips, 2304))
    for clip_index, start_sec in enumerate(range(0, int(self.video.duration), int(clip_duration))):
          end_sec = min(start_sec + clip_duration, self.video.duration)

          # Load the desired clip (replace with your actual clip loading logic)
          video_data = self.video.get_clip(start_sec=start_sec, end_sec=end_sec)

          # Apply the transform
          video_data = transform(video_data)

          # Move the inputs to the desired device
          inputs = video_data["video"]
          inputs = [i.to(device)[None, ...] for i in inputs]

          with torch.no_grad():
              features = self.model_feature_extraction_slowfast(inputs)
          # Convert the features to a NumPy array
          features = features.cpu().numpy()
          features = features.reshape(features.shape[0], -1)
          if (start_sec <2):
                print(features.shape)
                all_features[clip_index] = features
    return all_features
  def make_predictions(self,):
    scaler = StandardScaler()
    features = self.extract_features()
    print(len(features))
    features = scaler.fit_transform(features)
    predictions = self.MLP_Model.predict(features)
    Binary_predictions = (predictions> 0.5).astype("int32")
    return Binary_predictions
  def concatenated_clips(self, labels):
    clips = []
    temp_clips = []
    for count, prediction in enumerate(labels):
      if prediction == 1 or prediction == 1.0:
        # print('entered')
        temp_clips.append(count)
      else:
        clips.append(temp_clips)
        temp_clips = []
    if prediction == 1:
      clips.append(temp_clips)
      temp_clips = []
    print(temp_clips)
    clips = [sub_clip for sub_clip in clips if sub_clip]
    clips_by_one = clips
    tics_count =  sum(len(sub_array) for sub_array in clips)
    clips = [[sub_clip[0], sub_clip[-1]] for sub_clip in clips if sub_clip]
    print(clips)
    return clips, tics_count, clips_by_one
  def save_video(self, video_tensor, output_path, fps=30):
      video_tensor = video_tensor.permute(1, 2, 3, 0)
      io.write_video(output_path, video_tensor, fps)
  def Temporal_Detection(self, ):
      preds = self.make_predictions()
      preds = np.array(preds).reshape(-1)
      clips_in_ranges, tics_count, clips = self.concatenated_clips(preds)
      for i, (start, end) in enumerate(clips):
          video_tensor = self.video.get_clip(start, end)
          output_file = os.path.join(self.output_dir, f'clip_{i+1}.mp4')
          self.save_video(video_tensor['video'], output_file)
      return {'clips_in_ranges': clips_in_ranges, "output_path": self.output_dir, 'tics_count': tics_count, 'clips': clips, 'predictions':preds}

In [14]:
detect_tics = TemporalDetection('../downloaded_videos/subject1_video9.mp4',
                                '../uploads')


In [15]:
temporals = detect_tics.Temporal_Detection()

Video clips:  402
(1, 2304)
(1, 2304)
402
13/13 [==============================] - 0s 2ms/step
[]
[[2, 401]]


ValueError: too many values to unpack (expected 2)

In [ ]:
temporals

In [ ]:
def concateenated_clips(labels):
  clips = []
  temp_clips = []
  for count, prediction in enumerate(labels):
    if prediction == 1:
      temp_clips.append(count)
    elif prediction == 0:
      clips.append(temp_clips)
      temp_clips = []
  clips = [sub_clip for sub_clip in clips if sub_clip]
  clips = [[sub_clip[0], sub_clip[-1]] for sub_clip in clips if sub_clip]
  return clips

In [ ]:
video_path = '/content/drive/MyDrive/Tics Dataset/videos/extracted videos/subject1_video9.mp4'
preds = Temporal_Detection(video_path)

Video clips:  402
(1, 2304)
(1, 2304)
13/13 [==============================] - 0s 4ms/step


In [ ]:
clips = concateenated_clips(preds)

### Tell here the Temporal Method

In [ ]:
true = pd.read_csv('/content/drive/MyDrive/Tics Dataset/SlowfastTrial/1sec_Sampling/AllCombined/1_sec_data.csv')

In [ ]:
y_true = true[true['Video_number']==9]['labels']
y_true = y_true.map({'normal':0, 'tic':1})

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(preds.reshape(-1), y_true.values)

0.17164179104477612

In [ ]:
features = extract_features(video_path)

Video clips:  402
(1, 2304)


In [ ]:
# features.shape

(1, 1, 2304, 1, 1, 1)

In [ ]:
import glob
# glob.glob('/content/drive/MyDrive/Tics Dataset/SlowfastTrial/models/1 sec classification (tics temporal detection)/goodmodel.h5')
# tf.keras.models.load_model('/content/drive/MyDrive/Tics Dataset/SlowfastTrial/models/1 sec classification (tics temporal detection)/goodmodel.h5')

In [ ]:
# np.load('/content/drive/MyDrive/Tics Dataset/SlowfastTraial/1sec_Sampling/subject2_features/subject2_video0/patient3_video0_clip_0.npy').shape

(1, 2304)

In [ ]:
y_true.values

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
  def concateenated_clips( labels):
    clips = []
    temp_clips = []
    for count, prediction in enumerate(labels):
      if prediction == 1:
        temp_clips.append(count)
      elif prediction == 0:
        clips.append(temp_clips)
        temp_clips = []
    clips = [sub_clip for sub_clip in clips if sub_clip]
    clips_by_one = clips
    tics_count =  sum(len(sub_array) for sub_array in clips)
    clips = [[sub_clip[0], sub_clip[-1]] for sub_clip in clips if sub_clip]
    return clips, tics_count, clips_by_one

In [ ]:
clips, count, values = concateenated_clips([0,1,1,1,1,1,1,1])

[]


In [ ]:
clips, count, values

([[1, 7]], 7, [[1, 2, 3, 4, 5, 6, 7]])

In [ ]:
for i, val in enumerate(temporals['predictions'].tolist()):
  # print(val, val == 1)
  if val == 1:
    print(i)

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
